In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# NLTK tools for text processing
import re, nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'


import string
import spacy #used spacy for text prepocessing
import gensim
from gensim import corpora
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis #import pyLDAvis.gensim no longer works
import matplotlib.pyplot as plt
import seaborn as sns

C:\Users\helen\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
data = pd.read_csv('yelp_edmonton_reviews_restaurant_categories.csv')
data

C:\Users\helen\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,business_id,name,address,city,postal_code,latitude,longitude,stars,attributes,categories,user_id,review_stars,text,date
0,JX4tUpd09YFchLBuI43lGw,Naked Cyber Cafe & Espresso Bar,10303 108 Street NW,Edmonton,T5J 1L7,53.544682,-113.506589,4.0,"{'OutdoorSeating': 'False', 'BusinessParking':...","Arts & Entertainment, Music Venues, Internet S...",xTlqpuuqtO0FWs12gd1v6A,5,Came here to print off some documents and was ...,2014-10-15 01:49:26
1,JX4tUpd09YFchLBuI43lGw,Naked Cyber Cafe & Espresso Bar,10303 108 Street NW,Edmonton,T5J 1L7,53.544682,-113.506589,4.0,"{'OutdoorSeating': 'False', 'BusinessParking':...","Arts & Entertainment, Music Venues, Internet S...",f9lq4KAus-xCsmJmjXeKVw,3,Naked Cyber & Expresso Bar is a 24 hours cyber...,2010-01-26 21:14:29
2,JX4tUpd09YFchLBuI43lGw,Naked Cyber Cafe & Espresso Bar,10303 108 Street NW,Edmonton,T5J 1L7,53.544682,-113.506589,4.0,"{'OutdoorSeating': 'False', 'BusinessParking':...","Arts & Entertainment, Music Venues, Internet S...",r3QexFIhBXBT99canAgVEg,4,I visited Naked for the first time this weeken...,2013-02-05 17:01:52
3,JX4tUpd09YFchLBuI43lGw,Naked Cyber Cafe & Espresso Bar,10303 108 Street NW,Edmonton,T5J 1L7,53.544682,-113.506589,4.0,"{'OutdoorSeating': 'False', 'BusinessParking':...","Arts & Entertainment, Music Venues, Internet S...",xjafuSH_qDHXEkU0LyAxvg,4,Neat cafe! Delicious sandwiches and sweets wit...,2017-05-03 16:35:44
4,JX4tUpd09YFchLBuI43lGw,Naked Cyber Cafe & Espresso Bar,10303 108 Street NW,Edmonton,T5J 1L7,53.544682,-113.506589,4.0,"{'OutdoorSeating': 'False', 'BusinessParking':...","Arts & Entertainment, Music Venues, Internet S...",758g6NGLp9deCbvowz62Ww,3,"Naked is a cyber-cafe, one of the first ones t...",2008-10-22 02:56:43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56124,Rn3DtOiuJtc3whFySWXs5Q,Japanese Village,2940 Calgary Trail NW,Edmonton,T6J 7C2,53.463917,-113.494702,3.0,"{'RestaurantsAttire': ""u'casual'"", 'NoiseLevel...","Japanese, Restaurants, Steakhouses, Sushi Bars",jHtMpIDrHHVT1F4utXKFXQ,3,"We have been going to Japanese Village (""JV"") ...",2016-05-27 15:33:39
56125,Rn3DtOiuJtc3whFySWXs5Q,Japanese Village,2940 Calgary Trail NW,Edmonton,T6J 7C2,53.463917,-113.494702,3.0,"{'RestaurantsAttire': ""u'casual'"", 'NoiseLevel...","Japanese, Restaurants, Steakhouses, Sushi Bars",X8Z8Tax28_sAsPeNF922jw,2,Went Christmas eve as to celebrate holidays on...,2021-12-25 04:03:42
56126,Rn3DtOiuJtc3whFySWXs5Q,Japanese Village,2940 Calgary Trail NW,Edmonton,T6J 7C2,53.463917,-113.494702,3.0,"{'RestaurantsAttire': ""u'casual'"", 'NoiseLevel...","Japanese, Restaurants, Steakhouses, Sushi Bars",qQxRRzTT0U75A3mTVRh8xw,2,We came here a couple days ago for dinner. It ...,2016-05-16 12:16:46
56127,Rn3DtOiuJtc3whFySWXs5Q,Japanese Village,2940 Calgary Trail NW,Edmonton,T6J 7C2,53.463917,-113.494702,3.0,"{'RestaurantsAttire': ""u'casual'"", 'NoiseLevel...","Japanese, Restaurants, Steakhouses, Sushi Bars",f4B_zY-HuXqW00mWCzaclQ,1,I wouldn't recommend this restaurant. We went ...,2018-04-18 16:44:50


In [3]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

C:\Users\helen\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\helen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\helen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [34]:
def wordcount(data):
    comments = []
    stop_words = set(stopwords.words('english'))
    

    for words in data['text']:
        only_letters = re.sub("[^a-zA-Z]", " ",words)
        tokens = nltk.word_tokenize(only_letters) #tokenize the sentences
        lower_case = [l.lower() for l in tokens] #convert all letters to lower case
        filtered_result = list(filter(lambda l: l not in stop_words, lower_case)) #Remove stopwords from the comments
        comments.append(' '.join(filtered_result))
    
    
    w_string = []
    for t in data['text']:
        w_string.append(t)
    w_string = pd.Series(w_string).str.cat(sep=' ')
    
    
    co = CountVectorizer(ngram_range=(2,5))
    counts = co.fit_transform(comments)
    important_trigrams = pd.DataFrame(counts.sum(axis=0),columns=co.get_feature_names()).T.sort_values(0,ascending=False).head(50)
 
    important_trigrams=important_trigrams.reset_index()
    important_trigrams.rename(columns={'index':'word',0:'frequency'},inplace=True)
    return important_trigrams

C:\Users\helen\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [35]:
wordcount(data[data['name'] == 'Naked Cyber Cafe & Espresso Bar'])

C:\Users\helen\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,word,frequency
0,board games,5
1,computers available,4
2,cyber cafe,3
3,live music,3
4,drum kit,3
5,great customer service,2
6,customer service,2
7,stage drum kit,2
8,stage drum,2
9,chess table,2
